<a href="https://colab.research.google.com/github/ArpitaChatterjee/RNNs-Play-Generator/blob/main/RNN_Play_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Imports**

In [2]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


### **Datasets**

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [4]:
#Loading Your Own Data
from google.colab import files
path_to_file = list(files.upload().keys())[0]

IndexError: ignored

### **Read Contents of File**

In [5]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [7]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



### **Encoding**

In [8]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [9]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [10]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


### **Creating Training Examples**

In [11]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [13]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [14]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [15]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

## **Building the Model**

### Now it is time to build the model. We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes.

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, ##256
                              batch_input_shape=[batch_size, None]),##none==wedont know how long the sequences will be in each batch
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,#return intermidiate steps , otherwise model shows last output not inbtwn steps
                        stateful=True,
                        recurrent_initializer='glorot_uniform'), #what the values will take in lstm at start 
    tf.keras.layers.Dense(vocab_size) #final layer-> noofnodes==noofcharacter in the text
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


## **Creating a Loss Function**

In [17]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [18]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-1.89703819e-03 -5.89703955e-03 -2.00884975e-03 ... -7.52128690e-05
   -4.80467686e-03 -5.56411571e-04]
  [-3.99279204e-04 -6.33597793e-03  2.12126481e-03 ... -5.80863329e-04
    6.82282320e-04 -1.62663600e-05]
  [ 1.48616137e-03 -1.05754624e-03  8.67999811e-03 ...  6.24921056e-04
    7.11157976e-04  1.37365132e-03]
  ...
  [-2.03353423e-03 -8.39139987e-03  1.38957892e-02 ...  8.81619286e-03
   -1.42411806e-03  1.87050540e-03]
  [-1.15967426e-03 -2.24682782e-03  1.02629755e-02 ...  1.29282465e-02
   -2.53062695e-04  2.40949541e-03]
  [-1.05780887e-03 -7.89244287e-03  5.86100295e-03 ...  9.71492846e-03
   -5.48229972e-03  2.95950507e-04]]

 [[-2.22941558e-03  1.83767857e-04 -2.68761721e-03 ...  3.92024219e-03
   -1.45832298e-03  1.55246141e-03]
  [-8.38356931e-03  5.81982220e-03  3.66874365e-03 ...  5.72699076e-03
    9.30211041e-04  4.73548798e-03]
  [-9.05497558e-03  4.71149944e-03 -7.80117116e-04 ...  8.64975061e-03
   -5.01705334e-04  5.22391079e-03]
  ...
  [-9.078

In [19]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-1.89703819e-03 -5.89703955e-03 -2.00884975e-03 ... -7.52128690e-05
  -4.80467686e-03 -5.56411571e-04]
 [-3.99279204e-04 -6.33597793e-03  2.12126481e-03 ... -5.80863329e-04
   6.82282320e-04 -1.62663600e-05]
 [ 1.48616137e-03 -1.05754624e-03  8.67999811e-03 ...  6.24921056e-04
   7.11157976e-04  1.37365132e-03]
 ...
 [-2.03353423e-03 -8.39139987e-03  1.38957892e-02 ...  8.81619286e-03
  -1.42411806e-03  1.87050540e-03]
 [-1.15967426e-03 -2.24682782e-03  1.02629755e-02 ...  1.29282465e-02
  -2.53062695e-04  2.40949541e-03]
 [-1.05780887e-03 -7.89244287e-03  5.86100295e-03 ...  9.71492846e-03
  -5.48229972e-03  2.95950507e-04]], shape=(100, 65), dtype=float32)


In [20]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-1.8970382e-03 -5.8970395e-03 -2.0088498e-03 -1.7009813e-03
 -1.7281173e-03 -9.7410410e-04  3.6023695e-05 -1.9584175e-03
  5.7569034e-03 -7.3547419e-03 -3.6751518e-03  5.0564448e-04
 -7.3721055e-03  4.3338881e-04  1.7578256e-03 -1.0598394e-03
 -1.5078395e-03  2.5690396e-03 -1.4045745e-03  3.1659852e-03
  2.4116973e-03  6.0772789e-03 -2.0551322e-04 -4.1633910e-03
  7.7431863e-03 -2.0731594e-03 -4.3868129e-03 -6.8986439e-03
  2.8044730e-03  3.7375914e-03  1.2820869e-03  1.4251597e-03
 -1.2096168e-03 -7.5621400e-03 -1.6598463e-03 -1.0664815e-03
 -5.4423525e-03  5.2041872e-03  3.3196725e-03 -2.6109051e-03
 -1.0882326e-03 -8.9697284e-04 -3.9378689e-03 -1.5184967e-03
 -1.6085991e-03 -3.7351944e-03  5.1448029e-03 -1.4042285e-03
 -1.0214159e-03  2.9856840e-03  3.9570522e-03  4.9887509e-03
  3.3206402e-03  2.7489793e-04  3.2778794e-03  6.9128373e-04
  2.5028687e-03  1.5037522e-03 -3.9171795e-03  5.5924514e-03
 -4.2084986e-03 -2.1425807e-03 -7.5212869e-05 -4.8046769e-03
 -5.564115

In [21]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

';Zlmd:nFi--Vt$kPReFXxcP-S-UmA-dMy3Rdq :QM\n$tyWOfoMPWLvqml!ofQH:wAMuqkvPDCzD,NWjEqG:HCY!hqTRQwJ$dOBFX'

we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were.

In [22]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

## **Compiling the Model**

In [23]:
model.compile(optimizer='adam', loss=loss)

# Creating Checkpoints

In [24]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## **Training**

In [25]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 33s 171ms/step - loss: 3.0649
Epoch 2/50
172/172 [==============================] - 31s 172ms/step - loss: 2.0625
Epoch 3/50
172/172 [==============================] - 31s 172ms/step - loss: 1.7635
Epoch 4/50
172/172 [==============================] - 31s 171ms/step - loss: 1.5886
Epoch 5/50
172/172 [==============================] - 31s 170ms/step - loss: 1.4816
Epoch 6/50
172/172 [==============================] - 31s 170ms/step - loss: 1.4097
Epoch 7/50
172/172 [==============================] - 31s 170ms/step - loss: 1.3577
Epoch 8/50
172/172 [==============================] - 31s 170ms/step - loss: 1.3151
Epoch 9/50
172/172 [==============================] - 31s 171ms/step - loss: 1.2766
Epoch 10/50
172/172 [==============================] - 31s 171ms/step - loss: 1.2359
Epoch 11/50
172/172 [==============================] - 31s 170ms/step - loss: 1.1995
Epoch 12/50
172/172 [==============================] - 31s 170ms/step - lo

## Loading the Model

In [26]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Once the model is finished training, we can find the **lastest checkpoint** that stores the models weights using the following line.



In [27]:
#find the lastest checkpoint that stores the models weights 
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

We can load **any checkpoint** we want by specifying the exact file to load.

In [29]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

AttributeError: ignored

# **Generating Text**

In [30]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [31]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo
romeour mind.

HERMIONE:
No, by your leave:
I know you to a merry earth.

LORD BERKELEY:
My Lord, I have consider'd in my mind
With all prerogative: he is not there
Rut unknow the lords of England's raison.

ROMEO:
A gentleman of mild home.

GRUMIO:
What should it have to say?

LUCIO:
My lord, here comes my master, so blow
NGARET:
O, that they stay, then happiness is several tender champion
The purse thereof:
Ell if they bear me hence in Plantagenet, which haste, man. Friar Lodowick speak it,
As I intended of the lady-shining trembles:
broody children at our blood
Doth make me stay trutthen better than the melting news.

GLOUCESTER:
That you shall crim by you, it legs afroud
And nothing carent like to night.
I will my letter know your lordship came.

ISABELLA:
Anot a puissant man, I met the
dow
